# Scraping Transfermarkt

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Functions

In [2]:
def top_n_leagues(n):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

    page = "https://www.transfermarkt.co.uk/wettbewerbe/europa"
    pageTree = requests.get(page, headers=headers)
    pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

    LeagueList = []
    League = pageSoup.find_all("img", {"class": "continental-league-emblem"})

    if n > len(League):
        print("There is not that many leagues")
    else:
        for i in range(0,n):
            LeagueList.append(str(League[i]).split('" class',1)[0].split('<img alt="',1)[1])

    return LeagueList

In [3]:
def top_n_leagues_links(n):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

    page = "https://www.transfermarkt.co.uk/wettbewerbe/europa"
    pageTree = requests.get(page, headers=headers)
    pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

    LeagueList = []

    for name in n:
        League = pageSoup.find_all("a", {"title": name})
        for i in range(0,len(League)):
            LeagueList.append("https://www.transfermarkt.co.uk" + str(League[i]).split('" title', 1)[0].split('href="', 1)[1])

    return LeagueList

In [4]:
# Prints names only
def team_names(links):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    
    TeamList = []

    for link in links:
        page = link
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Team = pageSoup.find_all("img", {"class": "tiny_wappen"})

        for i in range(0,len(Team)):
            TeamList.append(str(Team[i]).split('" class',1)[0].split('<img alt="',1)[1])

    return TeamList

In [5]:
# Prints link only
def team_links(links, team_names):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    TeamList = []

    for link in links:
        page = link
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        for name in team_names:
            Team = pageSoup.find_all("a", {"title": name})
            x = 0
            for i in range(0,len(Team)):
                TeamList.append("https://www.transfermarkt.co.uk" + str(Team[i]).split('" title',1)[0].split('href="', 1)[1])
                x += 1
                if x > 0:
                    break

    return TeamList

In [6]:
def build_df(links):

    PlayersList = []
    AgeList=[]
    NationList=[]
    ValuesList = []
    PositionsList=[]
    ClubList = []

    for a in links:
        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
        page = a
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Players = pageSoup.find_all("img", {"class": "bilderrahmen-fixed lazy lazy"})
        
        Values = pageSoup.find_all("td", {"class": "rechts hauptlink"})

        for i in range(0,len(Values)):
            ValuesList.append(Values[i].text)

        for i in range(0,len(Players)):
            #print(str(Players[i]).split('" class',1)[0].split('<img alt="',1)[1], Values[i].text)
            PlayersList.append(str(Players[i]).split('" class',1)[0].split('<img alt="',1)[1])

    df= pd.DataFrame({"Players":PlayersList, "Values":ValuesList,})


    df.loc[(df.Values == '\xa0'),'Values']= None
        
    return df

# Premier League

In [11]:
leagues = top_n_leagues(10)
leagues = leagues[0:1]

league_links = top_n_leagues_links(leagues)

team_names = team_names(league_links)

every_team_links = team_links(league_links, team_names)
for i in range(len(every_team_links)):
    if not every_team_links[i].endswith('2023'):
        every_team_links[i] += '/saison_id/2023'

every_team_links = [s[:-1] + '1' for s in every_team_links]
every_team_links = every_team_links[0:20]

df = build_df(every_team_links)

df = df.dropna()
df

,Players,Values
0,Ederson,€45.00m
1,Zack Steffen,€6.00m
2,Scott Carson,€300k
4,Rúben Dias,€75.00m
5,Aymeric Laporte,€45.00m
...,...,...
797,Eddie Nketiah,€16.00m
798,Pierre-Emerick Aubameyang,€15.00m
799,Alexandre Lacazette,€15.00m
800,Folarin Balogun,€4.00m


# La Liga

In [12]:
leagues = top_n_leagues(10)
leagues = leagues[1:2]

league_links = top_n_leagues_links(leagues)

def team_names(links):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    
    TeamList = []

    for link in links:
        page = link
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Team = pageSoup.find_all("img", {"class": "tiny_wappen"})

        for i in range(0,len(Team)):
            TeamList.append(str(Team[i]).split('" class',1)[0].split('<img alt="',1)[1])

    return TeamList
team_names = team_names(league_links)

every_team_links = team_links(league_links, team_names)
for i in range(len(every_team_links)):
    if not every_team_links[i].endswith('2023'):
        every_team_links[i] += '/saison_id/2023'

every_team_links = [s[:-1] + '1' for s in every_team_links]
every_team_links = every_team_links[0:20]

df2 = build_df(every_team_links)

df2 = df2.dropna()
df2

,Players,Values
0,Thibaut Courtois,€60.00m
1,Andriy Lunin,€2.50m
2,Luis López,€300k
3,Toni Fuidias,€300k
5,Éder Militão,€60.00m
...,...,...
774,Mamadou Sylla,€1.20m
775,John Guidetti,€1.00m
776,Marc Tenas,€500k
777,Giorgi Gagua,€150k


# Serie A

In [13]:
leagues = top_n_leagues(10)
leagues = leagues[2:3]

league_links = top_n_leagues_links(leagues)

def team_names(links):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    
    TeamList = []

    for link in links:
        page = link
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Team = pageSoup.find_all("img", {"class": "tiny_wappen"})

        for i in range(0,len(Team)):
            TeamList.append(str(Team[i]).split('" class',1)[0].split('<img alt="',1)[1])

    return TeamList
team_names = team_names(league_links)

every_team_links = team_links(league_links, team_names)
for i in range(len(every_team_links)):
    if not every_team_links[i].endswith('2023'):
        every_team_links[i] += '/saison_id/2023'

every_team_links = [s[:-1] + '1' for s in every_team_links]
every_team_links = every_team_links[0:20]

df3 = build_df(every_team_links)

df3 = df3.dropna()
df3

,Players,Values
0,Alex Meret,€10.00m
1,David Ospina,€5.00m
2,Hubert Idasiak,€150k
3,Davide Marfella,€50k
5,Kalidou Koulibaly,€35.00m
...,...,...
822,Andrija Novakovich,€1.00m
823,Pietro Iemmello,€750k
824,Camillo Ciano,€500k
825,Alexander Satariano,€150k


# Bundasliga

In [14]:
leagues = top_n_leagues(10)
leagues = leagues[3:4]

league_links = top_n_leagues_links(leagues)

def team_names(links):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    
    TeamList = []

    for link in links:
        page = link
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Team = pageSoup.find_all("img", {"class": "tiny_wappen"})

        for i in range(0,len(Team)):
            TeamList.append(str(Team[i]).split('" class',1)[0].split('<img alt="',1)[1])

    return TeamList
team_names = team_names(league_links)

every_team_links = team_links(league_links, team_names)
for i in range(len(every_team_links)):
    if not every_team_links[i].endswith('2023'):
        every_team_links[i] += '/saison_id/2023'

every_team_links = [s[:-1] + '1' for s in every_team_links]
every_team_links = every_team_links[0:18]

df4 = build_df(every_team_links)

df4 = df4.dropna()
df4

,Players,Values
0,Manuel Neuer,€15.00m
1,Sven Ulreich,€900k
2,Christian Früchtl,€600k
3,Ron-Thorben Hoffmann,€500k
4,Lucas Hernández,€50.00m
...,...,...
631,Phillip Tietz,€2.00m
632,Luca Pfeiffer,€2.00m
633,Aaron Seydel,€600k
634,Braydon Manu,€550k


# Ligue 1

In [15]:
leagues = top_n_leagues(10)
leagues = leagues[4:5]

league_links = top_n_leagues_links(leagues)

def team_names(links):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    
    TeamList = []

    for link in links:
        page = link
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Team = pageSoup.find_all("img", {"class": "tiny_wappen"})

        for i in range(0,len(Team)):
            TeamList.append(str(Team[i]).split('" class',1)[0].split('<img alt="',1)[1])

    return TeamList
team_names = team_names(league_links)

every_team_links = team_links(league_links, team_names)
for i in range(len(every_team_links)):
    if not every_team_links[i].endswith('2023'):
        every_team_links[i] += '/saison_id/2023'

every_team_links = [s[:-1] + '1' for s in every_team_links]
every_team_links = every_team_links[0:18]

df5 = build_df(every_team_links)

df5 = df5.dropna()
df5

,Players,Values
0,Gianluigi Donnarumma,€50.00m
1,Keylor Navas,€8.00m
2,Sergio Rico,€4.00m
3,Alexandre Letellier,€400k
4,Garissone Innocent,€300k
...,...,...
655,Ibrahima Niane,€5.00m
656,Nicolas de Préville,€2.50m
657,Georges Mikautadze,€2.20m
658,Louis Mafouta,€900k


# Liga Portugal

In [17]:
leagues = top_n_leagues(10)
leagues = leagues[5:6]

league_links = top_n_leagues_links(leagues)

def team_names(links):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    
    TeamList = []

    for link in links:
        page = link
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Team = pageSoup.find_all("img", {"class": "tiny_wappen"})

        for i in range(0,len(Team)):
            TeamList.append(str(Team[i]).split('" class',1)[0].split('<img alt="',1)[1])

    return TeamList
team_names = team_names(league_links)

every_team_links = team_links(league_links, team_names)
for i in range(len(every_team_links)):
    if not every_team_links[i].endswith('2023'):
        every_team_links[i] += '/saison_id/2023'

every_team_links = [s[:-1] + '1' for s in every_team_links]
every_team_links = every_team_links[0:18]

df6 = build_df(every_team_links)

df6 = df6.dropna()
df6

,Players,Values
0,Odysseas Vlachodimos,€15.00m
1,Helton Leite,€3.00m
2,Mile Svilar,€2.00m
3,Leo Kokubo,€125k
4,Lucas Veríssimo,€10.00m
...,...,...
678,Pedro Henrique,€350k
679,Nikola Stojiljkovic,€300k
680,Paollo,€175k
681,Emerson Tucão,€75k


# Eredivisie

In [18]:
leagues = top_n_leagues(10)
leagues = leagues[6:7]

league_links = top_n_leagues_links(leagues)

def team_names(links):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    
    TeamList = []

    for link in links:
        page = link
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Team = pageSoup.find_all("img", {"class": "tiny_wappen"})

        for i in range(0,len(Team)):
            TeamList.append(str(Team[i]).split('" class',1)[0].split('<img alt="',1)[1])

    return TeamList
team_names = team_names(league_links)

every_team_links = team_links(league_links, team_names)
for i in range(len(every_team_links)):
    if not every_team_links[i].endswith('2023'):
        every_team_links[i] += '/saison_id/2023'

every_team_links = [s[:-1] + '1' for s in every_team_links]
every_team_links = every_team_links[0:18]

df7 = build_df(every_team_links)

df7 = df7.dropna()
df7

,Players,Values
0,André Onana,€12.00m
1,Jay Gorter,€1.50m
2,Przemyslaw Tyton,€300k
3,Remko Pasveer,€250k
4,Calvin Raatsie,€200k
...,...,...
677,Jens Odgaard,€2.00m
678,Roy Kuijpers,€350k
679,Finn Stokkers,€450k
680,Michiel Kramer,€375k


 # Süper Lig

In [21]:
leagues = top_n_leagues(10)
leagues = leagues[7:8]

league_links = top_n_leagues_links(leagues)

def team_names(links):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    
    TeamList = []

    for link in links:
        page = link
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Team = pageSoup.find_all("img", {"class": "tiny_wappen"})

        for i in range(0,len(Team)):
            TeamList.append(str(Team[i]).split('" class',1)[0].split('<img alt="',1)[1])

    return TeamList
team_names = team_names(league_links)

every_team_links = team_links(league_links, team_names)
for i in range(len(every_team_links)):
    if not every_team_links[i].endswith('2023'):
        every_team_links[i] += '/saison_id/2023'

every_team_links = [s[:-1] + '1' for s in every_team_links]
every_team_links = every_team_links[0:18]

df8 = build_df(every_team_links)

df8 = df8.dropna()
df8

,Players,Values
0,Altay Bayindir,€16.00m
1,Berke Özer,€3.50m
2,Bartu Kulbilge,€150k
3,Ertugrul Cetin,€75k
5,Min-jae Kim,€14.00m
...,...,...
719,Joel Pohjanpalo,€2.50m
720,Deniz Hümmet,€400k
721,Kubilay Kanatsizkus,€275k
722,Papiss Cissé,€250k


# Jupiler Pro League

In [22]:
leagues = top_n_leagues(10)
leagues = leagues[9:10]

league_links = top_n_leagues_links(leagues)

def team_names(links):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    
    TeamList = []

    for link in links:
        page = link
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Team = pageSoup.find_all("img", {"class": "tiny_wappen"})

        for i in range(0,len(Team)):
            TeamList.append(str(Team[i]).split('" class',1)[0].split('<img alt="',1)[1])

    return TeamList
team_names = team_names(league_links)

every_team_links = team_links(league_links, team_names)
for i in range(len(every_team_links)):
    if not every_team_links[i].endswith('2023'):
        every_team_links[i] += '/saison_id/2023'

every_team_links = [s[:-1] + '1' for s in every_team_links]
every_team_links = every_team_links[0:18]

df9 = build_df(every_team_links)

df9 = df9.dropna()
df9

,Players,Values
0,Simon Mignolet,€3.50m
1,Senne Lammens,€800k
2,Nick Shinton,€250k
4,Odilon Kossounou,€20.00m
5,Stanley Nsoki,€7.00m
...,...,...
645,Laurent Depoitre,€800k
646,Darko Lemajic,€650k
647,Yonas Malede,€500k
648,Ilombe Mboyo,€250k


# Final DataFrame

In [23]:
final_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9], ignore_index=True)

In [27]:
final_df.to_csv('transfermarkt_9_leagues_2021.csv', index = False)